# Excercise 2: Pre-processing and data augmentation

![img](http://www.paulvangent.com/files/DL_Course/day2_images/Ex2_intro.jpg)

Is this a male or female? You likely have no trouble recognizing this without effort. But what about a computer, could we teach it to recognize this? Of course we're talking biological, appearance based gender, not how a person chooses to identify. How someone identifies would be practically impossible even for us humans to determine based on a picture alone, let alone for a computer!

----

In this notebook we'll use the above classification task as a guiding example. We'll give you only a little data. The notebook will guide you through the pre-processing you can perform on your dataset prior to applying machine/deep learning. Pre-processing often can increase model performance significantly, and sometimes it is absolutely necessary for your models to learn anything at all.

There is an optional exercise at the end dealing with data augmentation. As we've not given you enough data to learn your model properly, what else can we do? This gives some ideas for what to do when you have little data. Sometimes you can create a 'virtual expansion' of your dataset by augmenting it with variations on existing data. This helps increase your dataset and helps create a more robust model in the end.

You will learn at least:
- Handling variable input sizes (2.1)
- Normalizing or standardizing your data (2.2)
- OPTIONAL: How can you augment your set to improve model performance (and should you)? (2.3)
- OPTIONAL: How do you split your data for training the deep learning model? What do you take into account (2.4)?

Let's get started!

-------------
    
**First, run the cell below to import the required packages we'll be using**

In [ ]:
#download required datasets for this notebook (might take a bit, be patient!)
from urllib.request import urlretrieve
import os
from zipfile import ZipFile

def download(url, file):
    if os.path.isfile(file):
        os.remove(file)
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")
        
def unzip(file):
    with ZipFile(file) as f:
        f.extractall()
    print('unzipped file: %s\n' %file)

try:
    download('http://www.paulvangent.com/files/DL_Course/celeba_larger.zip', 'celeba_larger.zip')
except:
    download('https://onedrive.live.com/download?cid=39383A5AFCD95065&resid=39383A5AFCD95065%21754594&authkey=AIcRTaFQqi0WnIo', 'celeba_larger.zip')
unzip('celeba_larger.zip')

try:
    download('http://www.paulvangent.com/files/DL_Course/misc_day2.zip', 'misc_day2.zip')
except:
    download('https://onedrive.live.com/download?cid=39383A5AFCD95065&resid=39383A5AFCD95065%21754608&authkey=AOJO5mcV9eCZsJ8', 'misc_day2.zip')
unzip('misc_day2.zip')

try:
    download('http://www.paulvangent.com/files/DL_Course/test_images.zip', 'test_images.zip')
except:
    download('https://onedrive.live.com/download?cid=39383A5AFCD95065&resid=39383A5AFCD95065%21754595&authkey=AETVSL8bPBIpTnI', 'test_images.zip')
unzip('test_images.zip')

import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import pandas as pd
import sys

from PIL import Image
import cv2
from scipy.ndimage import rotate
from keras.datasets import cifar10
from keras.utils import to_categorical
import keras

import utils_day2 as utils

## 2.1 - Variable input sizes

Usually the first step in any deep learning or machine learning project is getting your data ready. This means pre-processing your data and putting it in the appropriate container.

For these examples we'll use an excerpt of 2000 images from the CelebA dataset, which contains annotations for facial landmarks and a range of attributes, such as whether the person in the image is wearing a hat, has a beard, is attractive, etc.

For example, consider one random female and one random male entry from the set with attached annotated labels, and the annotated bounding box for the face overlaid:

![img](http://www.paulvangent.com/files/DL_Course/day2_images/celeba_1.jpg)
![img](http://www.paulvangent.com/files/DL_Course/day2_images/celeba_2.jpg)

However, before we can start training a model, there is a problem to fix. Every image in the set has different dimensions! While you can make a network work with variable input dimensions, because of the way the data is aggregated and passed through the network during training, at least the dimensions of all images in a given batch needs to be the same.

For now we'll train the network to handle images of a set size, because variable sizes will introduce other caveats that go beyond what we'll be doing today.

---------

**Exercise:**

Resize all images you loaded into memory to 100x100 pixels and put them into the numpy array we initialize below

- Try not to distort the images, think about how you can approach this and break into smaller problems
    
    hint:
    - the largest dimension needs to become 100
    - so you need to compute with what factor you need to multiply the dimensions so that the largest becomes 100 

[resizing hint](https://docs.opencv.org/3.0-alpha/modules/imgproc/doc/geometric_transformations.html?highlight=resize#cv2.resize), note the dimensions need to be integers

Second hint, a visualisation of what you need to do:
![resize and put in container](http://www.paulvangent.com/files/DL_Course/day2_images/Zero-pad.jpg)

In [ ]:
#get the data and labels
labels = pd.read_csv('celeba_larger/labels/annotations.txt')
#files are specified in the annotations
filelist = labels.values[:,0]
Y_data = labels['Male'].values

#first we define a numpy array to hold all image data
X_data= np.zeros(shape=(2000, 100, 100, 3), dtype=np.uint8)

for i in range(len(filelist)):
    img = cv2.imread('celeba_larger/data/%s' %filelist[i])
    
    ###Start of coding segment###
    
    #Complete the code below by following the steps outlined
    #get largest dimension
    largest_dim = 
    
    #compute with what factor we need to  multiply both dimensions
    resize_factor = 
    
    #compute new dimensions
    x_dim = 
    y_dim = 
    
    #resize image
    img_resized = cv2.resize(...)
    
    #put into container
    X_data[i][0:img_resized.shape[0], 0:img_resized.shape[1]] = img_resized
    
    ###End of coding segment###
    
#visualise a random image
plt.imshow(X_data[np.random.randint(0, len(X_data))][...,::-1])
plt.show()

## 2.2 - Normalizing or Standardizing your training data

We can almost start training our network! There is one last thing you usually do before training. In the exercises yesterday after loading the data, the first thing we did was normalize the input values. But why?

Remember from the lecture that for each layer in the network, its input is the output of the previous layer. Each layer will multiply its input with a series of weights, pass it through some activation function, and pass the output to the next layer.

**What do you think, could not normalizing the data be a problem? Why (not)?**

**Would the problem change if we use other data than images?**

-------------
**Exercise**

Let's run a little empirical exercise to warm up and see what happens when not normalizing and when normalizing. Let's try predicting gender from the CelebA subset you loaded and resized.

Complete the cell below to load up the convnet you designed yesterday and compile it.

- use binary crossentropy loss
- use the adamax optimizer


Hint: look at how we compiled the model yesterday, or [have a look at the Keras API](https://keras.io/models/model/#compile)

In [ ]:
#load the model
from misc import convnet
model = convnet.build_model(input_shape=(100, 100, 3), classes=1)

#split into training and validation set (more on this in 2.4)
X_train = X_data[0:1800]
X_val = X_data[1800:]

Y_train = Y_data[0:1800]
Y_val = Y_data[1800:]

###Start of coding segment###
#Add your code here
model.compile(...)

###End of coding segment###

print(model.summary())

**Exercise:**

Train the model you just compiled on non-normalized data for 5 epochs and evaluate the model performance. Let's start easy and see if we can teach the model to learn whether the image if male or female.

- Use X_train as training set and Y_train as training labels
- Shuffle the data between epochs
- Make sure to leave 'nonnorm_hist =' in front of the model.fit() call. We'll use this later on to plot progress
- 5 epochs should be enough, but feel free to play around!

Hint: look at how you did this yesterday, or [have a look at the Keras API](https://keras.io/models/model/#fit)

In [ ]:
#Add your code here (approx 1 line)
nonnorm_hist = model.fit(...)

- **Take a moment to look at the results. Describe what happened.**
- **Did the model learn anything at all? If so, are you satisfied with the results?**
- **Did the model learn anything at all?**

![img](http://www.paulvangent.com/files/DL_Course/day2_images/XKCD_shufflepile.png)

### Why Normalize your inputs?

Normalizing data can help your model optimizer to converge much better and faster to a global/local minimum. Without normalized inputs this can take a lot longer, or even prove impossible as you have just observed.

Let's confirm this with an empirical test. 

---------

**Exercise**

Complete the cell below with code to normalize the dataset, then run it to observe the behaviour of the same model on normalized data.

- Normalize all image data.
- Use same model compiling parameters.
- Run the model for 10 epochs (we suspect this one *just might* learn something..!).


In [ ]:
###Start of coding segment###
#Complete the code below to normalize the data (approx. 2 lines)
X_train_normalized = 
X_val_normalized = 
###End of coding segment###

#Re-initialize a blank model and compile it again
model = convnet.build_model(input_shape=(100, 100, 3), classes=1)

###Start of coding segment###
#Compile the model with binary crossentropy loss and adamax optimizer
...

#Complete the code below to fit the model for 10 epochs. Use batch size of 32
norm_hist = model.fit(...)
###End of coding segment

#Let's visualise the losses and accuracies of both sessions
print('\n\n' + ('-=' * 20))
print('performance of model on non-normalized data')
utils.plotter(nonnorm_hist)

print('performance of model on normalized data')
utils.plotter(norm_hist)

**What do you think, is it working well?**

**Can you see signs of overfitting?**

If you think you observe overfitting, or think the model needs to train longer, go back and change the number of epochs to a what you think a good estimate will be.

**Exercise:**

Once you're satisfied with the training result, run the cell below to test the model on a few images.

OPTIONAL: take a picture of yourself and upload to this notebook. On the expandable menu to the right you can upload the image from your computer. Put it in the folder 'test_images', then run the cell below.

If you don't want to test the model on yourself, just run the cell below anyway.

In [ ]:
images = glob('test_images/*.jpg')
testdata = utils.prepare_celeba_testdata(max_size=100, c_channels=3)
    
Y = model.predict(np.asarray(testdata))

for prediction, img in zip(Y, images):
    img = cv2.imread(img)
    genders = ['Female', 'Male']
    plt.title('I think this is a: %s' %(genders[int(round(prediction[0]))]))

    plt.imshow(img[...,::-1])
    plt.show()

**Can you explain what you've observed? Why would the behaviour be different with normalized input data?**

If you want, tweak the model parameters such as epochs, loss and optimizer and observe the effects. Can you improve the learning of the model?

### OPTIONAL: Should You Standardize Your Data?

Another pre-processing step you might come across is standardizing the data. This means scaling the data so that it conforms to a gaussian distribution with mean zero and standard deviation of 1. Whether standardizing is appropriate depends on your data, specifically its distribution.

**Can you think of the reason why?**

-------

As with many deep learning problems this is also an empirical one. Test the effects on model performance, does it improve with standardized data? Great, keep it! Note that normalization changes the numeric range without losing information (the distribution remains the same), whereas standardization changes the underlying distribution and can be sensitive to data containing outliers.

**Exercise:**

Standardise the images stored in variable *X_data*, and observe the effects of standardization on performance.

**Hint:** standardization is performed by the formula:

![standardization](http://www.paulvangent.com/files/DL_Course/day2_images/standardization.jpg)


In [ ]:
X_standardized = np.zeros((2000, 100, 100, 3), dtype=np.float32)

for i in range(len(X_data)):
    img = X_data[i]
    
    ###Start of coding segment
    #Standardize img
    
    
    
    ###End of coding segment###

    #We then put the standardized image into the array
    X_standardized[i] = img_standardized


#We initialize an empty model again and compile it
model = convnet.build_model(input_shape=(100, 100, 3), classes=1)
model.compile(loss='binary_crossentropy',
              optimizer='adamax',
              metrics=['accuracy'])

#Fit the model
std_hist = model.fit(X_standardized[:1800], Y_train,
                      validation_data=(X_standardized[1800:], Y_val),
                      epochs=10,
                      shuffle=True,
                      batch_size=32)

#Let's visualise the losses and accuracies of both sessions
print('\n\n' + ('-=' * 20))
print('performance of model on normalized data')
utils.plotter(norm_hist)

print('performance of model on standardized data')
utils.plotter(std_hist)

**What do you think, did it help?**

The end-state depends a bit on weight initialization and the path the optimizer takes, but in our case the maximum achieved accuracy was better on one set, whereas the loss was lower for the other set.

**Think about this. Which metric do you trust more? Why?** Ask us if you don't know, this is important!

----------

-----------------

## 2.3 Optional: Data Augmentation

Data augmentation, simply put, is creating extra training examples from your existing data. Consider the following examples of how you can create variations on the same image of a kitten.

![img](http://www.paulvangent.com/files/DL_Course/day2_images/kitten.jpg)

Run the following cell to render some variations on this image. Take notice of what we do numerically.

In [ ]:
download('http://www.paulvangent.com/files/DL_Course/day2_images/kitten.jpg', 'kitten.jpg')

img = np.array(Image.open('kitten.jpg'))


#Create a noise variant of the image
noise = np.random.randint(-100, 100, size=(img.shape[0], img.shape[1], 3))
#generate noise between 0.25 and 0.75
noise_amount = (0.75 - 0.25) * np.random.random_sample() + 0.25
#add noise to image
noisy_img = np.clip(img + (noise_amount * noise), a_min=0, a_max=255)
plt.title('added noise')
plt.imshow(np.uint8(noisy_img))
plt.show()


#Create a blurred variant of the image
blurred_img = cv2.blur(img, (7,7))
plt.title('blurred image')
plt.imshow(np.uint8(blurred_img))
plt.show()


#Rotate the image 45 degrees counter clockwise
rotated_img = rotate(img, 45, reshape=False)
plt.title('rotated image')
plt.imshow(np.uint8(rotated_img))
plt.show()

### So why would augmentation be useful?

Generally speaking data augmentation can create a more robust model. The reason is that we're increasing the variance in the dataset by transforming or distorting the images. Your model will be forced to learn robust descriptors of the data. For example, by adding a noisy, blurred and rotated version of all images to your set, your model will learn to deal with these transformations.

In many situations you may not have as much training data as you'd like. Using data augmentation in these cases can help your model learn more by providing it with more samples to learn from. 

### How to apply it?
You can write your own that does exactly what you want, which in some cases is preferable. However, Keras has a nice image data generator specifically for images that will do just fine in most situations. [Have a look at the Keras documentation here](https://keras.io/preprocessing/image/).

**Exercise:**

Apply the image data generator to the *X_train_normalized* and *X_val_normalized* set in memory, and train a model. [Look at the keras documentation for hints and examples](https://keras.io/preprocessing/image/).

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

###Start of coding segment###
datagen = ImageDataGenerator(...)
###End of coding segment###


model = convnet.build_model(input_shape=(100, 100, 3), classes=1)
model.compile(loss='binary_crossentropy',
              optimizer='adamax',
              metrics=['accuracy'])

###Start of coding segment###
model.fit_generator(...)
###End of coding segment###

### Should you apply it?
The main consideration for augmenting your dataset should be whether it increases performance. What do you want to predict? What are the boundary conditions of the data your model will encounter 'in the wild'? 

Say you want to predict someone's age from pictures of faces. If all the model will ever see at runtime is pictures with faces centered and horizontally aligned, adding rotated images to your training set makes little sense. 

The downside is your model needs to fit a more complex function to make predictions, which means you might need a bigger model that takes more memory and more time to compute. Think about what you want to achieve with it before choosing to augment your set or not.

**Too much augmentation can also cause underfitting (as you've problably observed)!**

-----------

## 2.4 OPTIONAL: Splitting Your Data Into Appropriate Sets

Remember from yesterday that it's common practice to split your dataset into a *training set*, *validation set* and *test set*. You use your *training set* to teach the model something, and monitor for signs of overfitting and other potential trouble by using the *validation set*. Once you finish tweaking the model and training, you use the last *test* holdout set to validate that the model indeed performs as you expect. How big should each set be? There is no magic number for this, but luckily there are some common sense rules of thumb we can follow.

For machine learning problems, where the datasets are usually a lot smaller, the [pareto principle](https://en.wikipedia.org/wiki/Pareto_principle) is often used, splitting the data 80/20 (train/test). For train/val/test sets a common distribution used to be 60/20/20 (train/val/test).

This is reasonable, as you need sufficient examples to evaluate the model's performance. For example, a common toy dataset used in machine learning is the [Iris dataset](https://scikit-learn.org/0.16/modules/generated/sklearn.datasets.load_iris.html) that contains data on how petal shapes of three iris plants relate to their species.

![img](http://www.paulvangent.com/files/DL_Course/day2_images/220px-Iris_versicolor_3.jpg)

With this set, one valid distribution could be 120 samples for training and 30 samples for testing, which gives the model enough data to learn from, and enough data for us to understand the performance. Testing a model on one or two examples will never give a good performance indication. 

**Can you explain why?**

-------------

### Moving to Deep Learning

With deep learning the data sets are usually **a lot** larger. 

![img](http://www.paulvangent.com/files/DL_Course/day2_images/bigdata.jpg)

There is no need to keep to the 80/20 or 60/20/20 conventions. This will create an unnecessarily large validation and test set, and more importantly, it deprives your model of a lot of training data. As a result, it might learn less than it otherwise would.

The bottom line is: set your validation and test sets to be large enough to give you enough confidence in the performance of your model.

For example, given a dataset of 1.000.000 examples, even a test set of 1% will give you 10.000 examples. This might very well be enough.

### Splitting smart

A particularly important consideration is not how large the split in your dataset is, but *how* you make the split. Consider you want to train a model to recognize cyclists and pedestrians. The first 10.000 images in your dataset are cyclists, the second 10.000 are pedestrians. You take the first 18.000 images (90%) to train your model. 1000(5%) are used for validation, and the last 1000 for testing. After training you find out:

- Performance on both the validation and test set is excellent (let's say 98.5% correct rate).
- When you test the system in real life, it turns out to not detect cyclists very well.

**What could be the problem here? Why didn't we catch this poor performance?**

**How would you solve this?**

---------

As you might have guessed from the example given, it is important that our validation and test sets are representative of the problem we want to solve. In the given example we only tested the model on pedestrian data since the set was ordered. This caused us to miss poor performance on detecting cyclists. This is important to check: many datasets are ordered by class. To solve this we simply shuffle the data before splitting it.

**Exercise:**

Shuffle the data below. [hint](https://pynative.com/python-random-shuffle/)

- Take note: you need to shuffle both the data and the labels the same way, or you'll lose the mapping between input and output data.

[the manual way hint](https://stackoverflow.com/a/23289591)

[the easy way hint](https://scikit-learn.org/stable/modules/generated/sklearn.utils.shuffle.html)

In [ ]:
import random
#first we load the cifar10 data again
(X_train, Y_train), (_, _) = cifar10.load_data()


###Start of coding segment###
#Take a look above at both ways to shuffle. Pick one and implement below.


###End of coding segment###

#let's test a few random entries in your shuffled data
#the labels in the output should match the pictures if you did it correctly
for i in np.random.randint(0, len(X_train), size=(5,)):
    plt.figure(figsize=(1,1))
    plt.imshow(X_train[i])
    plt.title(utils.c10_labels[int(Y_train[i])])
    plt.show()

The Keras .fit() function has a 'shuffle' flag. You can use this from now on, but now you know how to do it manually in case you need to shuffle your dataset before splitting into train/val/test sets.